<div align="center">
  <img src="logo_branding.png" width="250" alt="kavi.ai Logo">
  <h1>LoRA: Efficiency Without Compromise</h1>
  <p><b>A Premium Training Module by kavi.ai</b></p>
</div>

---

### 💎 **Smarter Overview**
Low-Rank Adaptation (LoRA) is an innovative paradigm that optimizes huge models by injecting rank-decomposition matrices, training only 1% of the total parameters.

### 🚀 **Enterprise Use Case**
Rapidly prototyping specialized agents for vertical SaaS where VRAM is a bottleneck.

### 📈 **Strategic Advantages**
- **Ultra-Efficient Storage**
- **No Latency Penalty**
- **Stability**

---

## Step 1: System Preparation

### **Purpose:**
Install the core PEFT and Transformers stack.

### **Line-by-Line Breakdown:**
- `peft`: The library that implements LoRA and other adapters.
- `bitsandbytes`: For memory-efficient training.

In [ ]:
!pip install transformers --upgrade
!pip install datasets
!pip install trl[peft] --upgrade
!pip install -U git+https://github.com/huggingface/trl
!pip install bitsandbytes loralib
!pip install wandb -U
!pip install hf_transfer


In [ ]:
%env HF_HUB_ENABLE_HF_TRANSFER=True
%env WANDB_PROJECT=LLM-Training-Course
%env WANDB_RUN_ID=LORA
%env WANDB_NOTEBOOK_NAME={__vsc_ipynb_file__}

In [ ]:
import wandb
wandb.login()

In [ ]:
import sys
sys.path.append('/root/llm-training-course/')

In [ ]:
from datasets import load_dataset

train_ds, eval_ds = load_dataset("mlabonne/orpo-dpo-mix-40k", split=["train[:10%]","train[10%:15%]"])

In [ ]:
train_ds

In [ ]:
train_ds = train_ds.map(lambda x: { "messages": [{"role":"system", "content": x["prompt"] }] + x["chosen"] })
eval_ds = eval_ds.map(lambda x: { "messages": [{"role":"system", "content": x["prompt"] }] + x["chosen"] })

In [ ]:
columns_to_remove = [c for c in train_ds.column_names if c not in ["messages"]]
train_ds = train_ds.remove_columns(columns_to_remove)

columns_to_remove = [c for c in eval_ds.column_names if c not in ["messages"]]
eval_ds = eval_ds.remove_columns(columns_to_remove)

## Step 2: Model Loading

### **Purpose:**
Load the base model (e.g., Llama-3) in bfloat16 for efficiency.

### **Line-by-Line Breakdown:**
- `torch_dtype=torch.bfloat16`: Reduces memory usage while maintaining precision.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

In [ ]:
tokenizer.chat_template

In [ ]:
tokenizer.eos_token_id

In [ ]:
tokenizer.eos_token = "<|eot_id|>"
tokenizer.eos_token_id = tokenizer.convert_tokens_to_ids(tokenizer.eos_token)

In [ ]:
tokenizer.eos_token_id

In [ ]:
print(tokenizer)
print("---")
print("Vocab size:", tokenizer.vocab_size)
print("---")
print("Chat template:", tokenizer.chat_template)

In [ ]:
chat_template = open('../chat_templates/llama-3-chat.jinja').read()
chat_template = chat_template.replace('    ', '').replace('\n', '')
print("Chat Template", chat_template)
tokenizer.chat_template = chat_template
print("---")
print(tokenizer.apply_chat_template(train_ds["messages"][0], tokenize=False))

In [ ]:
from helpers import set_padding_for_tokenizer
set_padding_for_tokenizer(tokenizer)

In [ ]:

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

In [ ]:
print(tokenizer.vocab_size)
len(tokenizer)

In [ ]:
print(model)

In [ ]:
from helpers import stream_responses_for_sample
from transformers import GenerationConfig

generation_config =  GenerationConfig(max_new_tokens=50)

sample_conversations = [
    [{"role": "user", "content": "What is the capital of France?"}],
    [{"role": "user", "content": "Write me a javascript function that check if string is palindrome."}],
    [{"role": "user", "content": "Given x^2=36-4 what is x?"}]
]
stream_responses_for_sample(model, tokenizer, sample_conversations, generation_config=generation_config)

## Step 3: Configure LoRA Adapters

### **Purpose:**
Define the structure of the low-rank matrices to be injected.

### **Line-by-Line Breakdown:**
- `r`: The rank of the decomposition (lower = fewer parameters).
- `target_modules`: Which layers to apply LoRA to (usually attention projections).

In [ ]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                     "gate_proj", "up_proj","down_proj"],
    modules_to_save=["embed_tokens", "input_layernorm", "post_attention_layernorm"],
    bias="none",
    task_type="CAUSAL_LM"
)

peft_model = get_peft_model(model, peft_config)

In [ ]:
peft_model


In [ ]:
import os
from trl import SFTConfig, SFTTrainer

args = SFTConfig(
    output_dir=os.getenv("WANDB_RUN_ID"),
    report_to="wandb",
    num_train_epochs=1.0,
    do_train=True,
    do_eval=True,
    log_level="debug",
    gradient_checkpointing=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    per_device_eval_batch_size=1,
    lr_scheduler_type="constant",
    bf16=True,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=0.1,
    max_grad_norm=.3,
    learning_rate=1e-4,
)


## Step 4: Execute Training

### **Purpose:**
Run the optimization loop.

### **Line-by-Line Breakdown:**
- `trainer.train()`: Updates ONLY the LoRA parameters.

In [ ]:
trainer = SFTTrainer(
    model=peft_model,
    tokenizer=tokenizer,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds
)
trainer.train()

In [ ]:
from transformers import GenerationConfig
generation_config =  GenerationConfig(max_new_tokens=50, eos_token_id=tokenizer.eos_token_id)

sample_conversations = [
    [{"role": "user", "content": "What is the capital of France?"}],
    [{"role": "user", "content": "Write me a javascript function that check if string is palindrome."}],
    [{"role": "user", "content": "Given x^2=36-4 what is x?"}]
]
stream_responses_for_sample(peft_model, tokenizer, sample_conversations, generation_config=generation_config)